In [1]:
import json
import pandas as pd

from sklearn.model_selection import train_test_split

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from tensorflow.keras.layers import (Embedding,
                                     LSTM,
                                     Dense,
                                     Dropout,
                                     GlobalMaxPool1D,
                                     BatchNormalization)
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot


In [2]:
#opening already translated and tagged dataset
with open('translated_dataframe.json', 'r', encoding='utf-8') as file:
    translated_data = json.load(file)

In [3]:
translated_df = pd.DataFrame(translated_data)

In [4]:
translated_df.head()

,original,translated,tag
0,rearrange closet,Перестановление шкафа,1
1,meeting tasks,Встреча задач,2
2,taste of home,Вкус дома,1
3,sociology paper,Социологический документ,3
4,osteopath,остеопат,1


In [5]:
X = translated_df['translated']
y = translated_df['tag']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:
#finding the number of unique words - vocabulary size
def num_words(sentence):
  words_list = []
  for sent in sentence:
    for word in sent.split():
      if word.lower() not in words_list:
        words_list.append(word.lower())
      else:
        pass
  return words_list

evrth_list = []
for sentence in translated_df['translated']:
    evrth_list.append(sentence)

    
num_words = num_words(evrth_list)
len(num_words)

1620

In [7]:
#finding the longest and the shortest sentence
count_list = []
for sentence in translated_df['translated']:
    splitted = sentence.split()
    count_list.append(len(splitted))

print(max(count_list))
print(min(count_list))

10
1


In [8]:
#onehot encodiding and embedding to turn text into numbers and vectors
vocabulary = 1700
sentence = 10
def embedding(X):
    onehot_vector = [one_hot(words, vocabulary) for words in X]
    embedded = pad_sequences(onehot_vector, padding="post", maxlen=sentence)
    return embedded

In [9]:
final_data = embedding(X_train)
final_data[100:120]

array([[  32,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [1428, 1110,  893,    0,    0,    0,    0,    0,    0,    0],
       [ 594,  880,    0,    0,    0,    0,    0,    0,    0,    0],
       [ 704,  125,    0,    0,    0,    0,    0,    0,    0,    0],
       [1289,  159,    0,    0,    0,    0,    0,    0,    0,    0],
       [1468,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [1067,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [ 384,  862,    0,    0,    0,    0,    0,    0,    0,    0],
       [1016,  385,    0,    0,    0,    0,    0,    0,    0,    0],
       [1077, 1490,    0,    0,    0,    0,    0,    0,    0,    0],
       [ 425,  144,  447,    0,    0,    0,    0,    0,    0,    0],
       [ 682,  597,    0,    0,    0,    0,    0,    0,    0,    0],
       [ 605,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [ 695,  564,    0,    0,    0,    0,    0,    0,    0,    0],
       [  96,  851, 1667,    0,   

In [10]:
#same for test data
test_data = embedding(X_test)

In [11]:
#lstm model
def model():
  vector_features = 32
  lstm_model = Sequential()
  lstm_model.add(Embedding(vocabulary,
                      vector_features,
                      input_length=sentence))
  lstm_model.add(LSTM(100, return_sequences = True))
  lstm_model.add(GlobalMaxPool1D())
  lstm_model.add(BatchNormalization())
  lstm_model.add(Dropout(0.5))
  lstm_model.add(Dense(10, activation="relu"))
  lstm_model.add(Dropout(0.25))
  lstm_model.add(Dense(1, activation = "sigmoid"))
  return lstm_model

In [14]:
lstm_model = model()
lstm_model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
lstm_model.summary()

TypeError: VariableMetaclass._variable_v1_call() got an unexpected keyword argument 'experimental_enable_variable_lifting'

In [23]:
fitted = lstm_model.fit(final_data, y_train, epochs=8, batch_size=32)

Epoch 1/8


UnimplementedError: Graph execution error:

Detected at node 'binary_crossentropy/Cast' defined at (most recent call last):
    File "<frozen runpy>", line 198, in _run_module_as_main
    File "<frozen runpy>", line 88, in _run_code
    File "C:\Users\189\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\189\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "C:\Users\189\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 725, in start
      self.io_loop.start()
    File "C:\Users\189\AppData\Roaming\Python\Python311\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\189\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_events.py", line 607, in run_forever
      self._run_once()
    File "c:\Users\189\AppData